In [ ]:
# The following code is a classification neural network

# Import all packages and libraries for NN

# from numpy.random import seed
# seed(1)
# from tensorflow import set_random_seed
# set_random_seed(2)

from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import shuffle
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import argmax
from keras.utils import to_categorical

import numpy
import keras
import pandas


In [ ]:
# Data Processing
# Preparing Iris dataset for usage as a dataframe
# Encoding output variables
# Shuffle rows randomly for variety
# Spliting dataset to test and train dataframes. Test dataset will be 80% of entire dataframe.
# Parsing dataframe to features and labels (x_train and y_train)

# Constants
_training_split = 0.8


# Spliting dataframe to training and testing
def split_to_training(dataframe):
    train_df = dataframe[:int(len(dataframe)*_training_split)]
    
    return train_df


def split_to_testing(dataframe):
    test_df = dataframe[int(len(dataframe)*_training_split):]
    
    return test_df


# Spliting training or testing dataset to x and y
def split_to_x(dataframe):
    x = dataframe[features].values
    
    return x


def split_to_y(dataframe):
    y = dataframe[labels].values
    
    return y


# Encoding all labels with HOT Encoder
def encode_dataframe(label_y):
    encoder = LabelEncoder()
    encoder.fit(label_y)
    encoded_Y = encoder.transform(label_y)
    new_y = np_utils.to_categorical(encoded_Y)
    
    return new_y


def prep_data(df):
    pass
    
# Load in dataset
dataframe = pandas.read_csv('iris.csv')
df = shuffle(dataframe)
features = list(df.columns.values)[:-1]
labels = list(df.columns.values)[-1]
input_dim = len(list(df.columns.values)[:-1])

train_df = split_to_training(df)
x_train = split_to_x(train_df)
y_train = encode_dataframe(split_to_y(train_df))

test_df = split_to_testing(df)
x_test = split_to_x(test_df)
y_test = encode_dataframe(split_to_y(test_df))



In [3]:
class KerasNeuralNetwork:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
        
    def run(self):
        _number_of_test_models = 5
        
        base_number_layers = int(input_dim**.5)
        list_of_loss = []
        list_of_models = []
        list_of_history = []
        for i in range(_number_of_test_models):
            model = Sequential()
            # Base number of layers
            model.add(Dense(input_dim, input_dim=input_dim, activation='sigmoid'))
            for i in range(base_number_layers):
                model.add(Dense(5, activation='sigmoid'))
            model.add(Dense(3, activation='sigmoid'))
            model.summary()
        
            model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
            list_of_models.append(model)

            
            history = model.fit(self.x_train, self.y_train, epochs=30)
            list_of_history.append(history)
        
        
            # Compare loss after each epoch
            loss_after_each_epoch = history.history['loss']
            avg_roc = abs((loss_after_each_epoch[0]-loss_after_each_epoch[-1])/len(loss_after_each_epoch))
            list_of_loss.append(avg_roc)
            base_number_layers += 1
            
            scores = model.evaluate(self.x_test, self.y_test)
            print("{}: {}".format(model.metrics_names[0], scores[0]))
            print("{}: {}%".format(model.metrics_names[1], scores[1]*100))
        
        # Take the first trough in list and use that model
        minimal_loss_position = 0;
        for index, value in enumerate(list_of_loss):
            if value < list_of_loss[index+1]:
                minimal_loss_position = index
                break
        print(list_of_loss)
        print(list_of_models)
        print('The optimal model is at index: {}'.format(minimal_loss_position))
        optimized_model = list_of_models[minimal_loss_position]
        optimized_history = list_of_history[minimal_loss_position]
        
        # Evaluating the model
        print(optimized_history.history)
        scores = optimized_model.evaluate(self.x_test, self.y_test)
        print("{}: {}".format(optimized_model.metrics_names[0], scores[0]))
        print("{}: {}%".format(optimized_model.metrics_names[1], scores[1]*100))

        

In [4]:
NN = KerasNeuralNetwork(x_train, y_train, x_test, y_test)
NN.run()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 25        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 18        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
120/120 [==============================] - 0s 3ms/step - loss: 0.4681 - acc: 0.3333
Epoch 2/30
120/120 [==============================] - 0s 98us/step - loss: 0.4668 - acc: 0.3333
Epoch 3/30
120/120 [==============================] - 0s 100us/step - loss: 0.4654 - ac

Epoch 1/30
120/120 [==============================] - 1s 6ms/step - loss: 0.4967 - acc: 0.3333
Epoch 2/30
120/120 [==============================] - 0s 154us/step - loss: 0.4957 - acc: 0.3333
Epoch 3/30
120/120 [==============================] - 0s 121us/step - loss: 0.4947 - acc: 0.3333
Epoch 4/30
120/120 [==============================] - 0s 129us/step - loss: 0.4937 - acc: 0.3333
Epoch 5/30
120/120 [==============================] - 0s 136us/step - loss: 0.4927 - acc: 0.3333
Epoch 6/30
120/120 [==============================] - 0s 135us/step - loss: 0.4917 - acc: 0.3333
Epoch 7/30
120/120 [==============================] - 0s 149us/step - loss: 0.4908 - acc: 0.3333
Epoch 8/30
120/120 [==============================] - 0s 128us/step - loss: 0.4898 - acc: 0.3333
Epoch 9/30
120/120 [==============================] - 0s 131us/step - loss: 0.4888 - acc: 0.3333
Epoch 10/30
120/120 [==============================] - 0s 118us/step - loss: 0.4878 - acc: 0.3333
Epoch 11/30
120/120 [==========

Epoch 1/30
120/120 [==============================] - 1s 9ms/step - loss: 0.5270 - acc: 0.3333
Epoch 2/30
120/120 [==============================] - 0s 132us/step - loss: 0.5257 - acc: 0.3333
Epoch 3/30
120/120 [==============================] - 0s 145us/step - loss: 0.5244 - acc: 0.3333
Epoch 4/30
120/120 [==============================] - 0s 150us/step - loss: 0.5231 - acc: 0.3333
Epoch 5/30
120/120 [==============================] - 0s 143us/step - loss: 0.5218 - acc: 0.3333
Epoch 6/30
120/120 [==============================] - 0s 139us/step - loss: 0.5205 - acc: 0.3333
Epoch 7/30
120/120 [==============================] - 0s 137us/step - loss: 0.5192 - acc: 0.3333
Epoch 8/30
120/120 [==============================] - 0s 146us/step - loss: 0.5180 - acc: 0.3333
Epoch 9/30
120/120 [==============================] - 0s 152us/step - loss: 0.5167 - acc: 0.3333
Epoch 10/30
120/120 [==============================] - 0s 134us/step - loss: 0.5154 - acc: 0.3333
Epoch 11/30
120/120 [==========